# Trajectory optimization for a 2 link planar manipulator


Import statements, check that casadi symbolic variables are working

In [2]:
import numpy as np
from casadi import *
x = MX.sym("x")
print(jacobian(sin(x),x))


cos(x)


Build a class for storing all manipulator properties and EoMs

In [25]:
## Class for Manipulator Dynamics
class Manipulator:

    def __init__(self, M1, M2, L1, L2, theta1, theta2):

        # Manipulator parameters
        self.M1 = M1
        self.M2 = M2
        self.L1 = L1
        self.L2 = L2

        # Define simulation parameters
        self.T = 30                                         # not sure what a reasonable amount is
        self.dt = 0.01
        self.n = self.T/self.dt
        self.g = 9.81

        # Define arrays for system matrices
        self.M = MX.sym('M', 4,4,self.n)
        self.c = MX.sym('c', 2,1,self.n)
        self.G = MX.sym("G", 2,1,self.n)

        # Define our decision variables
        self.X = MX.sym('X', 4, self.n)           # X = [theta1, theta2, theta1_dot, theta2_dot].T
        self.tau = MX.sym('tau', self.n)          # tau = [tau1, tau2].T

    def M(self):
        # Sets up the array of mass matrices
        for k in range(0, self.n):
            [theta1, theta2, theta1dot, theta2dot] = [self.X[0, k], self.X[1, k], self.X[2, k], self.X[3, k]]
            self.M[1,1,k] = (self.M1 + self.M2) * self.L1**2
            self.M[1,2,k] = self.M2 * self.L1 * self.L2 * cos(theta1 - theta2)
            self.M[2,1,k] = self.M2 * self.L1 * self.L2 * cos(theta1 - theta2)
            self.M[2,2,k] = self.M2 * self.L2**2
        return self.M

    def c(self):
        for k in range(0, self.n):
            [theta1, theta2, theta1dot, theta2dot] = [self.X[0, k], self.X[1, k], self.X[2, k], self.X[3, k]]
            self.c[1,1,k] = self.M2*self.L1*self.L2*theta2dot**2 * sin(theta1-theta2)
            self.c[2,1,k] = -self.M2*self.L1*self.L2*theta1dot**2 * sin(theta1-theta2)
        return self.c

    def G(self):
        for k in range(0, self.n):
            [theta1, theta2, theta1dot, theta2dot] = [self.X[0, k], self.X[1, k], self.X[2, k], self.X[3, k]]
            self.G[1,1,k] = (self.M1+self.M2) * self.g * self.L1 * cos(theta1)
            self.G[2,1,k] = self.M2 *self.g *self.L2 * cos(theta2)
        return self.G

    def Xdot(self):
        # Returns the time derivative of X as defined in the doc
        # TODO
        return Xdot

    def build_equality_constraints(X_start, X_finish, T, dt):
        # Build constraints as defined in the doc
        return eq_constraints

    

IndentationError: expected an indented block after function definition on line 23 (164745986.py, line 26)

In [31]:
# Just using this cell to check if casadi behaves the way i think it will
a = MX.sym('a',4,4)
b = MX.sym('b',4,1)
for i in range(0,4):
    ai = MX.sym("ai",0,4)
    ai[0:4] = a[:, i]*i
    b[:,i] = ai
print(b)

RuntimeError: .../casadi/core/slice.cpp:80: Assertion "stop<=len" failed:
Slice (start=0, stop=4, step=1) out of bounds with supplied length of 0

In [ ]:
#SX for scalar operations, MX for Matrix operations
# https://web.casadi.org/docs/#a-simple-test-problem For optimization stack

#direct shooting vs direct multiple shooting vs indirect vs all at once

NLP solvers available:
blocksqp
bonmin
ipopt
knitro
snopt
worhp
feasiblesqpmethod
qrsqp
scpgen
sqpmethod

QP solvers:
clp
cplex
fatrop
gurobi
highs
hpipm
hpmpc
ooqp
osqp
proxqp
qpoases
sqic
superscs
ipqp
nlpsol
qrqp

In [ ]:
opti = casadi.Opti()

x = opti.variable()
y = opti.variable()

opti.minimize(  (y-x**2)**2   )
opti.subject_to( x**2+y**2==1 )
opti.subject_to(       x+y>=1 )

opti.solver('ipopt')


sol = opti.solve()

print(sol.value(x))
print(sol.value(y))

# opti.set_initial(sol1.value_variables())


In [ ]:
#Lagrange multiplers (dual variables)
con = sin(x+y)>=1
opti.subject_to(con)
sol = opti.solve()

print(sol.value(opti.dual(con)))
